# HW01
* 이름: 아무개
* 학번: 99990000

위에 이름과 학번을 자신의 것으로 기입하고 또 맨 위에 노트북의 제목도 `HW01<이름><학번>`으로 바꾸어 HiClass를 통해 마감시간까지 제출하면 됩니다.
마감시간은 HiClass에 공지되어 있습니다.

In [28]:
class RGB {
    int r, g, b;
    RGB() { r=0; g=0; b=0; }
    RGB(int _r, int _g, int _b) { r=_r; g=_g; b=_b; }
    @Override public String toString() { return "rgb("+r+","+g+","+b+")"; }
}

com.twosigma.beaker.javash.bkrcb76a6d3.RGB

In [29]:
class Point {
    int x, y;
    Point() { x=0; y=0; }
    Point(int _x, int _y) { x=_x; y=_y; }
    @Override public String toString() { return "("+x+","+y+")"; }
}

com.twosigma.beaker.javash.bkrcb76a6d3.Point

## 첫번째 설계
* `CircObj`, `RectObj`, `RTriObj` 클래스 작성 (3점)
* 같은 클래스의 인스턴스끼리 (`CircObj`끼리, `RectObj`끼리, `RTriObj`) 충돌 검사 테스트 사례 작성 및 각 테스트 사례를 svg 출력으로 확인 (3점)
* 서로 다른 클래스의 (`CircObj`와 `RectObj`, `RectObj`와 `RTriObj`, `RTriObj`와 `CircObj`) 인스턴스 충돌 검사 테스트 사례 작성 및 각 테스트 사례를 svg 출력으로 확인 (3점)

In [30]:
abstract class ShapeObj { // 어떤 추상적인 모양의 물건 클래스
    Point p;   // 물건 위치에 대한 정보
    RGB color; // 물건의 색깔
    ShapeObj(Point p, RGB c) { this.p=p; color=c; }
    abstract Point closest(Point p); // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로) //원 내부는 자기자신
    abstract boolean collides(Point p); // 점 p와 충돌하는가?
    abstract boolean collides(ShapeObj o); // o와 충돌하는가?
    abstract String toSVG(); // svg 태그 생성
    @Override
    public String toString() { return toSVG(); }
}

com.twosigma.beaker.javash.bkrcb76a6d3.ShapeObj

<pre>
<h2> 원에서의 점과의 충돌 검출 방법 </h2>
원과 점의 closest 및 collides는 중점에서 모든 방향의 거리가 같다는 것을 이용하여, 
해당 점과의 거리가 반지름보다 긴지를 체크하여 쉽게 구할 수 있다.
<img src="https://github.com/sbj1229/oop2019Spring/raw/master/circle2.png"/> 
closest은 점을 반환하는데 원이 점 밖에 있는경우 도형에 가장 인접한 점을 반환해야 하므로
삼각형의 닮은 성질을 이용하여 구할 수 있다.
<img width="200" height="200" src="https://github.com/sbj1229/oop2019Spring/raw/master/circle.png"/> 
<h2> 원에서의 도형과의 충돌 검출 방법 </h2>
먼저  상대 도형에서 내 중심과 가장 가까운 점을 구한 뒤,
구한 점이 내 도형과 충돌하는지 검사한다.
아래 그림은 테스트 결과이다.
<img width="400" height="200" src="https://github.com/sbj1229/oop2019Spring/raw/master/circle3.png"/> 

</center>

In [31]:
class CircObj extends ShapeObj { // 원 모양 클래스
    int r;
    CircObj(int r, Point p, RGB c) { super(p, c); this.r = r; }

    @Override
    Point closest(Point p) {
        int x = this.p.x - p.x; 
        int y = this.p.y - p.y;
        double len = Math.sqrt(x*x+y*y); // 두 점의 거리를 구한다.
        if(len <= r) return p; // 반지름이 길이보다 크거나 같은 경우, 점이 원 내부나 경계에 존재하므로 해당 점을 리턴
            
        // 반지름이 길이보다 작은경우, 직긱삼각형 비례를 이용하여 x,y 좌표를 구하여 리턴한다.
        int dx = (int)Math.round( (p.x-this.p.x)*r/len ); 
        int dy = (int)Math.round( (p.y-this.p.y)*r/len );
        return new Point(this.p.x+dx, this.p.y+dy);
    }
    @Override
    boolean collides(Point p) {
        int x = this.p.x - p.x; 
        int y = this.p.y - p.y;
        double len = Math.sqrt(x*x+y*y);
        return len <= r; // 반지름이 길이보다 크거나 같은 경우, 점이 원 내부에 존재하므로 충돌
    }
    @Override
    boolean collides(ShapeObj o) { return collides(o.closest(p)); }
    @Override
    String toSVG() {
        return "<circle cx='"+this.p.x+"' cy='"+this.p.y+"' r='"+r+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.CircObj

<pre><h2>직사각형에서의 점과의 충돌 검출 방법</h2>
직사각형은 가로/세로가 각각 x,y축과 평행하다는 제약조건이 있으므로,
대각선/곡선에 따른 추가 연산 필요 없이 단순 비교 연산으로 충돌 여부를 구할수 있다.
<img src="https://github.com/sbj1229/oop2019Spring/raw/master/rect.png"/>
점의 x,y를 px,py라 하고 사각형의 점을 rx,ry라 칭하면,

(1) px가 rx보다 작고, py가 ry보다 작다면      rx,ry     값을 반환 
(2) px가 rx~rx+w 사이이고, py가 ry보다 작다면 px,ry     값을 반환 
(3) px가 rx+w보다 크고, py가 ry보다 작다면    rx+w,ry   값을 반환 
(4) px가 rx보다 크고, py가 ry~ry+h 사이라면   rx+w,py 값을 반환 
(5) px가 rx보다 크고, py가 ry보다 크다면      rx+w,ry+h 값을 반환 
(6) px가 rx~rx+w 사이이고, py가 ry보다 크다면 rx,py   값을 반환 
(7) px가 rx보다 작고, py가 ry보다 크다면      rx,ry+h   값을 반환
(8) px가 rx보다 작고, py가 ry~ry+h 사이라면   rx,py   값을 반환 
(9) px가 rx~rx+w 사이이고,  py가 ry~ry+h 사이라면  p를 반환

점과의 충돌검사는 위의 조건식처럼 점이 (9) 안에 있는지를 rx <= px <=rx+w && ry <= py <= ry+h 조건식으로 검사하면 된다.

<h2>직사각형에서의 도형과의 충돌 검출 방법</h2>
직사각형에서는 아래와 같은 3가지 경우의 충돌이 있을 수 있는데,
<img width="600" height="300" src="https://github.com/sbj1229/oop2019Spring/raw/master/rect2.png"/>

1)의 해결방법은 상대 도형의 중점이 내 도형과 충돌하는지 검사하는 것
2)의 해결방법은 내 꼭짓점 4개를 상대 도형과 충돌하는지 검사하는 것
3)의 해결방법은 내 변의 좌표를 따라 반복하면서 상대 도형과 충돌하는지 검사하는 것

여기서 2)의 해법은 3)에 해법에 포함되는데,
2)의 해법은 O(4) 의 실행시간을 가지지만 
3)의 해법은 O(n)의 실행시간을 가지므로 먼저 검사한다.

In [32]:
class RectObj extends ShapeObj { 
    int w; // 너비 (양수)
    int h; // 높이 (양수)
    RectObj(int w, int h, Point p, RGB c) {super(p,c); this.w = w; this.h = h;}
    
    @Override
    Point closest(Point p) 
    {
        if (collides(p)) return p; // 내부에 포함되는 점일 때
        
        if(p.x < this.p.x   && p.y < this.p.y  ) return new Point(this.p.x,   this.p.y  ); // 왼쪽   위   꼭지점 
        if(p.x < this.p.x   && p.y > this.p.y+h) return new Point(this.p.x,   this.p.y+h); // 왼쪽   아래 꼭지점
        if(p.x > this.p.x+w && p.y > this.p.y+h) return new Point(this.p.x+w, this.p.y+h); // 오른쪽 아래 꼭지점 
        if(p.x > this.p.x+w && p.y < this.p.y  ) return new Point(this.p.x+w, this.p.y  ); // 오른쪽 위   꼭지점
        
        if(p.x < this.p.x && (this.p.y <= p.y && p.y <= this.p.y+h))   // 왼쪽 변 바깥
            return new Point(this.p.x, p.y);
        if((this.p.x <= p.x && p.x <= this.p.x+w) && p.y > this.p.y+h) // 아래쪽 변 바깥
            return new Point(p.x, this.p.y+h);
        if(p.x > this.p.x+w && (this.p.y <= p.y && p.y <= this.p.y+h)) // 오른쪽 변 바깥
            return new Point(this.p.x+w, p.y);
        if((this.p.x <= p.x && p.x <= this.p.x+w) && p.y < this.p.y)   // 위쪽 변 바깥
            return new Point(p.x, this.p.y);
        
        System.err.println("RectObj closest: impossible happened");
        return null; // 여기에 결코 오면 안된다
    }
    @Override
    boolean collides(Point p) {
        return this.p.x <= p.x && p.x <= this.p.x+w
            && this.p.y <= p.y && p.y <= this.p.y+h;
    }
    @Override
    boolean collides(ShapeObj o) { 
        if(collides(o.p)) return true; // 상대 도형 중심이 내 안에 들어와 있으면 충돌
            
        if (o.collides(this.p) ||   // 각 꼭지점 충돌 검사
            o.collides(new Point(this.p.x+w, this.p.y  )) ||
            o.collides(new Point(this.p.x,   this.p.y+h)) ||
            o.collides(new Point(this.p.x+w, this.p.y+h)) ) 
            return true;
        
        // 각 변 충돌 검사
        for(int x=this.p.x; x<=this.p.x+w; x++) {
            if(o.collides(new Point(x, this.p.y))) return true;
            if(o.collides(new Point(x, this.p.y+h))) return true;
        }
        for(int y=this.p.y; y<=this.p.y+h; y++) {
            if(o.collides(new Point(this.p.x, y))) return true;
            if(o.collides(new Point(this.p.x+w, y))) return true;
        }
        
        return false; // 경계에서도 충돌이 발생하지 않으면 미충돌
    }
    @Override
    String toSVG() {
        return "<rect x='"+this.p.x+"' y='"+this.p.y+"' width='"+w+"'height='"+h+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.RectObj

In [33]:
import com.twosigma.beakerx.widget.*;

RTriObj t1 = new RTriObj(50, 50,new Point( 80,80 ), new RGB(255,0,0));
RectObj r1 = new RectObj(100,100, new Point( 100,100 ), new RGB(0,0,255));
RectObj r2 = new RectObj(30,60, new Point( 110,190 ), new RGB(0,255,255));
CircObj c1 = new CircObj(30, new Point( 220,150), new RGB(0,255,0) );

CircObj c3 = new CircObj(5, r1.closest(t1.p), new RGB(255,0,0) );
CircObj c4 = new CircObj(5, r1.closest(r2.p), new RGB(0,255,255) );
CircObj c5 = new CircObj(5, r1.closest(c1.p), new RGB(0,255,0) );

System.out.println("사각형과 사각형 충돌검사 : " +r1.collides(r2));
System.out.println("사각형과 삼각형 충돌검사 : " +r1.collides(t1));
System.out.println("사각형과 원 충돌검사 : " +r1.collides(c1));

HTML widget = new HTML();
String sgv = 
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + r1 + r2 + c1 + c3 + c4 + c5 + 
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");
return widget;

사각형과 사각형 충돌검사 : true
사각형과 삼각형 충돌검사 : true
사각형과 원 충돌검사 : true


<pre><h2>직각삼각형에서의 점과의 충돌 검출 방법</h2>
직각삼각형의 경우 이미 만들었던 사각형을 이용하여 쉽게 구할 수 있다.

<img src="https://github.com/sbj1229/oop2019Spring/raw/master/triangle.png"/>
closest의 경우 삼각형과의 충돌검사를 시행하여 충돌할 경우 내부이므로 (1)이 해결되고 점 p를 반환한다.
아니라면 사각형을 생성하여 점 p와 사각형의 closest 을 구한 뒤, 해당 점이 삼각형과 겹친다면 삼각형의 직선쪽이므로 해당 점을 반환하고 (2)가 해결된다.
겹치지 않는다면, (3)의 영역이므로 빗변의 점들과 p를 모두 비교해서 가장 거리가 짧은 점을 반환한다.

collides의 경우 역시 사각형과 collides을 구한뒤, 해당 결과가 거짓이면 사각형 밖이므로 거짓이고,
삼각형의 높이나 너비가 0인경우는 사각형과 동일하므로(대각선 미존재) 사각형의 collides와 동일하다.
두 경우가 모두 아닐경우, 그림의 (3)의 영역중 사각형 내부 이므로, 
<img src="https://github.com/sbj1229/oop2019Spring/raw/master/triangle2.png"/>
0 <= t1 <= 1 
0 <= t2 <= 1 
t1 + t2 <= 1
의 세 백터 조건을 모두 만족하면 내부이다.
그러나 우리가 작성한 직각삼각형은 너비와 높이가 독립적인 수치이므로, 백터가 아닌 좌표값으로만 본다면 각 항에 너비/높이를 곱해준 
아래 식을 모두 만족한다면 내부이다.
0 <= t1 <= w 
0 <= t2 <= h 
t1*h + t2*w <= w*h 


<h2>직각삼각형에서의 도형과의 충돌 검출 방법</h2>

직각 삼각형 역시 직사각형과 동일한 충돌 검출 방식을 사용하는데,
1)상대 도형의 중점이 내 도형과 충돌하는지 검사하는 것
2)내 꼭짓점 3개를 상대 도형과 충돌하는지 검사하는 것
3)내 변의 좌표를 따라 반복하면서 상대 도형과 충돌하는지 검사하는 것

In [34]:
// 삼각형 연산용 
class Calc{
    static Point Shortlen(Point a,Point c,Point p) // 선분 ac와 점 p의 가장 가까운 접점은 어디인가?
    {
            int x = a.x - c.x;
            int y = a.y - c.y;
            double px = c.x, py = c.y;
            double min = 9999999; // 초기값이므로 크게
            double minx=0,miny=0;
            double length = Math.sqrt(x*x+y*y);
            double xr = x/length; // 대각선 점들의 위치변화값
            double yr = y/length;
            double ix,iy,il;
            while((px >= Math.min(a.x,c.x) && px<= Math.max(a.x,c.x)) && (py >= Math.min(a.y,c.y) && py<=Math.max(a.y,c.y)) )
            { // px,py가 대각선의 시작/도착점 범위 안에서만 돌도록
                px += xr; //px,py : 현재 검사하고 있는 점. 초기값은 대각선 변의 끝점중 하나
                py += yr; 
                ix=px-p.x; // 검사할 점과 현재 대각선 위의 점의 좌표차이 
                iy=py-p.y;
                il = Math.sqrt(ix*ix+iy*iy); // 거리
                if(il<min){ // 현재 위치가 더 가깝다면 거리와 좌표를 기록
                    min = il;
                    minx = px;
                    miny = py;
                }
            }
            Point mins = new Point((int)Math.round(minx),(int)Math.round(miny)); // 반복문의 결과인 거리가 가장 짧은 점을 리턴
            return mins;
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.Calc

In [35]:
class RTriObj extends ShapeObj {
    int w; // 직각인 꼭지점으로부터 음수/양수로 왼쪽/오른쪽 방향으로의 w만큼 떨어진 꼭지점 나타냄
    int h; // 직각인 꼭지점으로부터 음수/양수로 아래쪽/위쪽 방향으로의 h만큼 떨어진 꼭지점 나타냄
    
    RTriObj(int w, int h, Point p, RGB c) {
        super(p,c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point p) 
    {              
        if(collides(p)) return p; // 내부
        
        // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        Point a  = new Point(this.p.x+w, this.p.y); // 꼭지점 
        Point c  = new Point(this.p.x, this.p.y+h); 
        int minX = Math.min(this.p.x, this.p.x+w);  
        int minY = Math.min(this.p.y, this.p.y+h);

        Point p4 = new RectObj(Math.abs(w), Math.abs(h), new Point(minX, minY), color).closest(p); // 사각형 closest를 돌려서
        if (collides(p4)) return p4; // 그게 삼각형에 있는 점이면 그걸 그대로 사용하면 됨
        
        // 아니라면 나머지 경우는 빗변들 따라 돌리는 경우밖에 남지 않는다
        return Calc.Shortlen(a,c,p);
    }
    @Override
    boolean collides(Point p) {
        int minX = Math.min(this.p.x, this.p.x+w); 
        int minY = Math.min(this.p.y, this.p.y+h);
        boolean collidesRect = new RectObj(Math.abs(w), Math.abs(h), new Point(minX, minY), color).collides(p);
        // 포함하는 직사각형 안에 안들어오면 가망이 없다
        if (!collidesRect) return collidesRect;
        // 높이나 너비가 0이면 직사각형과 다를 바 없다
        if (h==0 || w==0) return collidesRect;
        
        // (p.x, p,y) = (this.p.x + t1*w , this.p.y + t2*h)
        // 0 <= t1 <= 1, 0 <= t2 <= 1, t1+t2 <= 1 을 확인
        int t1w = p.x - this.p.x; // 중점과 x,y축 차
        int t2h = p.y - this.p.y;
        return ((w>0)  ? 0 <= t1w && t1w <= w : 0 >= t1w && t1w >= w) // 차이가 높이/너비보다 직고
            && ((h>0)  ? 0 <= t2h && t2h <= h : 0 >= t2h && t2h >= h)
            && ((w*h>0)? t1w*h + t2h*w <= w*h : t1w*h + t2h*w >= w*h); // 차이의 합이 0~w*h범위 안쪽일 때
    }
    @Override
    boolean collides(ShapeObj o) 
    {
        Point a = new Point (this.p.x+w, this.p.y); // x축 변화
        Point c = new Point (this.p.x, this.p.y+h); // y축 변화
        
        if (collides(o.p)) return true; // 상대 도형의 기준점이 들어와 있다면 충돌

        if (o.collides(this.p) || o.collides(a) || o.collides(c)) return true; // 각 꼭지점 충돌 검사

        int maxX = Math.max(this.p.x, this.p.x+w),minX = Math.min(this.p.x, this.p.x+w); 
        int maxY = Math.max(this.p.y, this.p.y+h),minY = Math.min(this.p.y, this.p.y+h);
        // 축에 평행한 변의 점들에 대해 충돌검사
        for(int x=minX; x<=maxX; x++) if(o.collides(new Point(x, this.p.y))) return true;
        for(int y=minY; y<=maxY; y++) if(o.collides(new Point(this.p.x, y))) return true;
        
        // 가로 세로 중 더 긴 쪽을 기준으로 1씩 변화시키며 빗변을 따라 충돌검사
        if( Math.abs(w) >= Math.abs(h) ) {
            for(int x=minX; x<=maxX; x++) // 가로가 길면 x값 1씩 변화
                if(o.collides(new Point(x, a.y + (int)Math.round(h*(a.x-x)/w))))
                    return true;
        } else {
            for(int y=minY; y<=maxY; y++) // 세로가 길면 y값 1씩 변화
                if(o.collides(new Point(c.x + (int)Math.round(w*(c.y-y)/h), y)))
                    return true;
        }

        return false;
    }    
        
    @Override
    String toSVG() { 
        return "<polygon points='"+this.p.x+","+this.p.y+" "+ (this.p.x+w)+","+this.p.y +" "+this.p.x+","+(this.p.y +h)+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }

}

com.twosigma.beaker.javash.bkrcb76a6d3.RTriObj

공식 출처 :  https://mathjk.tistory.com/176 <br>
이런식으로 넓이를 구해서 삼각형의 내부 연산 하는 등 방법은 여러가지, 위의 예시는 대각선/곡선에 대해 비교적 안정적인 케이스<br>
<code>static boolean EqArea(Point a,Point b,Point c,Point p) // 넓이를 이용하여 점이 삼각형 내부에 위치했는지
{
    int abcarea = Math.round( Math.abs( (a.x-b.x)*c.y + (b.x-c.x)*a.y + (c.x-a.x)*b.y) / 2);
    double abparea =  Math.abs((a.x-b.x)*p.y + (b.x-p.x)*a.y + (p.x-a.x)*b.y) / 2) ;
    double acparea = Math.abs((a.x-p.x)*c.y + (p.x-c.x)*a.y + (c.x-a.x)*p.y) / 2) ;
    dobule bcparea =  Math.abs((p.x-b.x)*c.y + (b.x-c.x)*p.y + (c.x-p.x)*b.y) / 2) ;
    return abcarea == Math.round(abparea + acparea + bcparea );
}</code>

In [36]:
import com.twosigma.beakerx.widget.*;
String s= TestDataObj.Vaild();
HTML widget = new HTML();
widget.setValue("<div>"+s+"</div>");
return widget;

## 두번째 설계
첫번째 설계와 같은 문제를 해결하지만 조금 다른 방식의 설계로 프로그램을 재작성해 보라.
* `Circle`, `Rectangle`, `RTriangle` 클래스를 작성하라 (1점)
* 첫번째에서 했던 같은 테스트 사례들로 모두 실행해 보라 (3점)

In [37]:
abstract class Shape { // 어떤 추상적인 모양 클래스
    RGB color; // 물건의 색깔
    Shape(RGB c) { color=c; }
    abstract String toSVG(Point self);
    abstract Point closest(Point self, Point p);
    abstract boolean collides(Point self, Point p);
    abstract boolean collides(Point self, Obj o);
}

class Obj {
    Point p;
    Shape s; // Shape이 Obj의 속성이다
    Obj(Point _p, Shape _s) { p=_p; s=_s; }
    Point closest(Point p) { return s.closest(this.p, p); } // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
    boolean collides(Point p) { return s.collides(this.p, p); } // 점 p와 충돌하는가?
    boolean collides(Obj o) { return s.collides(this.p, o); } // o와 충돌하는가
    @Override
    public String toString() { return s.toSVG(this.p); }
}

com.twosigma.beaker.javash.bkrcb76a6d3.Shape

In [38]:
import com.twosigma.beakerx.widget.*;

RTriObj t1 = new RTriObj(-100,-100,new Point( 250,250 ), new RGB(255,0,0));
RectObj r1 = new RectObj(100,100, new Point( 50,50 ), new RGB(0,0,255));
CircObj c1 = new CircObj(50, new Point( 150,150), new RGB(0,255,0) );


RTriangle t2 = new RTriangle( -100,-100, new RGB(255,0,0) );
Rectangle r2 = new Rectangle( 100,100, new RGB(0,0,255) );
Circle c2 = new Circle( 50 ,new RGB(0,255,0) );
Obj o1 = new Obj(new Point( 250,250), t2);
Obj o2 = new Obj(new Point( 50,50 ), r2);
Obj o3 = new Obj(new Point( 150,150 ), c2);

HTML widget = new HTML();
String sgv = 
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + r1 + c1 +
    "</svg>" +
    "<svg width='300' height='300' style='border:solid 1px'>"
    + o1 + o2 + o3 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");
return widget;

In [39]:
class Circle extends Shape { // 원 모양 클래스
    int r;
    Circle(int r, RGB c) { super(c); this.r = r; }

    @Override
    Point closest(Point self, Point p) {
        int x = self.x - p.x; 
        int y = self.y - p.y;
        double len = Math.sqrt(x*x+y*y); // 두 점의 거리를 구한다.
        if(len <= r)  return p; // 반지름이 길이보다 크거나 같은 경우, 점이 원 내부나 경계에 존재하므로 해당 점을 리턴
    
        // 반지름이 길이보다 작은경우, 직긱삼각형 비례를 이용하여 x,y 좌표를 구하여 리턴한다.
        int dx = (int)Math.round( (p.x-self.x)*r/len );
        int dy = (int)Math.round( (p.y-self.y)*r/len );
        return new Point(self.x+dx, self.y+dy);
    }

    @Override
    boolean collides(Point self, Point p) { 
        int x = self.x - p.x; 
        int y = self.y - p.y;
        double len = Math.sqrt(x*x+y*y); // 두 점의 길이를 구한다.
        return len <= r; // 반지름이 길이보다 크거나 같은 경우, 점이 원 내부에 존재하므로 충돌
    }
    @Override
    boolean collides(Point self, Obj o) {
        return collides(self, o.closest(self));
    }
    @Override
    String toSVG(Point self) {
        return "<circle cx='"+self.x+"' cy='"+self.y+"' r='"+r+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.Circle

In [40]:
import com.twosigma.beakerx.widget.*;
HTML widget = new HTML();

Obj t1 = new Obj(new Point(150,150) , new Circle(70,new RGB(0,255,255))); // 70, 100 & 100 , 170

Point p1 =  new Point(80,80);

Circle c1  = new Circle(40 , new RGB(0,255,0));
Circle c2 = new Circle(5,  new RGB(255,0,0));

Obj o1 = new Obj(p1,c1);
Obj o2 = new Obj(t1.closest(p1),c2);
Obj o3 = new Obj(o1.closest(new Point(150,150)),c2);

String sgv = "";
sgv += "<p>초록색 원의 중점과 하늘색 원의 closest : "+o2.p+"</p>";
sgv +="<p>closest의 좌표와 초록색 원의 충돌검사  : "+o1.collides(o2.p)+"</p>";
sgv +=
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o2 +
    "</svg>";

sgv += "<p>하늘색 원의 중점과 초록색 원의 closest : "+o3.p+"</p>";
sgv +="<p>closest의 좌표와 하늘색 원의 충돌검사  : "+t1.collides(o3.p)+"</p>";
sgv +=
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o3 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");

return widget; 

In [41]:
class Rectangle extends Shape { 
    int w; // 너비 (양수)
    int h; // 높이 (양수)
    Rectangle(int w,int h, RGB c) {super(c); this.w = w; this.h = h;}
    
    @Override
    Point closest(Point self, Point p) 
    {
        if (collides(self, p)) return p;

        if(p.x < self.x   && p.y < self.y  ) return new Point(self.x, self.y);     // 왼쪽 위 꼭지점
        if(p.x < self.x   && p.y > self.y+h) return new Point(self.x, self.y+h);   // 왼쪽 아래 꼭지점
        if(p.x > self.x+w && p.y > self.y+h) return new Point(self.x+w, self.y+h); // 오른쪽 아래 꼭지점
        if(p.x > self.x+w && p.y < self.y  ) return new Point(self.x+w, self.y);   // 오른쪽 위 꼭지점
        
        if(p.x < self.x && (self.y <= p.y && p.y <= self.y+h)) // 왼쪽 변 바깥
            return new Point(self.x, p.y);
        if((self.x <= p.x && p.x <= self.x+w) && p.y > self.y+h) // 아래쪽 변 바깥
            return new Point(p.x, self.y+h);
        if(p.x > self.x+w && (self.y <= p.y && p.y <= self.y+h)) // 오른쪽 변 바깥
            return new Point(self.x+w, p.y);
        if((self.x <= p.x && p.x <= self.x+w) && p.y < self.y) // 위쪽 변 바깥
            return new Point(p.x, self.y);
        
        System.err.println("Rectangle closest: impossible happened");
        return null; // 여기에 결코 오면 안된다
    }
    @Override
    boolean collides(Point self, Point p) {
        return self.x <= p.x && p.x <= self.x+w
            && self.y <= p.y && p.y <= self.y+h;
    }
    @Override
    boolean collides(Point self, Obj o) { 
        if(collides(self, o.p)) return true; // 상대 도형 중심이 내 안에 들어와 있으면 충돌
            
        if (o.collides(self) ||   // 각 꼭지점 충돌 검사
            o.collides(new Point(self.x+w, self.y  )) ||
            o.collides(new Point(self.x,   self.y+h)) ||
            o.collides(new Point(self.x+w, self.y+h)) ) 
            return true;
        
        // 각 변 충돌 검사
        for(int x=self.x; x<=self.x+w; x++) {
            if(o.collides(new Point(x, self.y))) return true;
            if(o.collides(new Point(x, self.y+h))) return true;
        }
        for(int y=self.y; y<=self.y+h; y++) {
            if(o.collides(new Point(self.x, y))) return true;
            if(o.collides(new Point(self.x+w, y))) return true;
        }
            
        return false; // 경계에서도 충돌이 발생하지 않으면 미충돌
    }
    @Override
    String toSVG(Point self) {
        return "<rect x='"+self.x+"' y='"+self.y+"' width='"+w+"'height='"+h+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.Rectangle

In [42]:
import com.twosigma.beakerx.widget.*;
Obj t1 = new Obj(new Point(100,100) , new Rectangle(100,100,new RGB(0,255,255))); // 70, 100 & 100 , 170

Point p1 =  new Point(50,50);
Point p2 =  new Point(50,230);
Point p3 =  new Point(230,230);
Point p4 =  new Point(230,50);
Point p5 =  new Point(90,120);

Rectangle c1  = new Rectangle(30,30 , new RGB(255,0,0));
Rectangle c2  = new Rectangle(30,30  ,new RGB(255,100,0)); 
Rectangle c3  = new Rectangle(30,30  ,new RGB(255,255,0));
Rectangle c4  = new Rectangle(30,30  ,new RGB(0,255,0));
Rectangle c5  = new Rectangle(30,30  ,new RGB(255,0,255));

Circle c6  = new Circle(5,  new RGB(255,0,0));
Circle c7  = new Circle(5,  new RGB(255,100,0)); 
Circle c8  = new Circle(5,  new RGB(255,255,0));
Circle c9  = new Circle(5,  new RGB(0,255,0));
Circle c10 = new Circle(5,  new RGB(255,0,255)); 


Obj o1 = new Obj(t1.closest(p1),c6);
Obj o2 = new Obj(t1.closest(p2),c7);
Obj o3 = new Obj(t1.closest(p3),c8);
Obj o4 = new Obj(t1.closest(p4),c9);
Obj o5 = new Obj(t1.closest(p5),c10);
Obj o6 = new Obj(p1,c1);
Obj o7 = new Obj(p2,c2);
Obj o8 = new Obj(p3,c3);
Obj o9 = new Obj(p4,c4);
Obj o10 = new Obj(p5,c5);


System.out.println("하늘색 사각형과 빨주노초 사각형의 충돌검사 or : " +(o9.collides(t1) || o6.collides(t1) || o7.collides(t1) || o8.collides(t1)));
System.out.println("하늘색 사각형과 자주색 사각형의 충돌검사 : " +o10.collides(t1));

HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o2 + o3 + o4 + o5 + o6 + o7 + o8 + o9 + o10 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; 

하늘색 사각형과 빨주노초 사각형의 충돌검사 or : false
하늘색 사각형과 자주색 사각형의 충돌검사 : true


In [43]:
class RTriangle extends Shape {
    int w; // 직각인 꼭지점으로부터 음수/양수로 왼쪽/오른쪽 방향으로의 w만큼 떨어진 꼭지점 나타냄
    int h; // 직각인 꼭지점으로부터 음수/양수로 아래쪽/위쪽 방향으로의 h만큼 떨어진 꼭지점 나타냄
    
    RTriangle(int w, int h, RGB c)  
    {
        super(c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point self, Point p) {
        if(collides(self, p)) return p; // 내부
        
        // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        Point a = new Point (self.x+w,self.y); // x축 변화
        Point c = new Point (self.x,self.y+h); // y축 변화
        int maxX = Math.max(self.x, self.x+w),minX = Math.min(self.x, self.x+w); 
        int maxY = Math.max(self.y, self.y+h),minY = Math.min(self.y, self.y+h);
        
        Point p4 = new Rectangle(Math.abs(w), Math.abs(h), color).closest(new Point(minX, minY), p); // 사각형 closest를 돌려서
        if (collides(self, p4)) return p4; // 그게 삼각형에 있는 점이면 그걸 그대로 사용하면 됨

        return Calc.Shortlen(a,c,p);

    }
    @Override
    boolean collides(Point self, Point p) {
        int minX = Math.min(self.x, self.x+w); 
        int minY = Math.min(self.y, self.y+h);
        boolean collidesRect = new Rectangle(Math.abs(w), Math.abs(h), color).collides(new Point(minX, minY), p);
        // 포함하는 직사각형 안에 안들어오면 가망이 없다
        if (!collidesRect) return collidesRect;
        // 높이나 너비가 0이면 직사각형과 다를 바 없다
        if (h==0 || w==0) return collidesRect;
        
        // (p.x, p,y) = (self.x + t1*w + self.y + t2*h)
        // 0 <= t1 <= 1, 0 <= t2 <= 1, t1+t2 <= 1 을 확인
        int t1w = p.x - self.x;
        int t2h = p.y - self.y;
        return ((w>0)  ? 0 <= t1w && t1w <= w : 0 >= t1w && t1w >= w)
            && ((h>0)  ? 0 <= t2h && t2h <= h : 0 >= t2h && t2h >= h)
            && ((w*h>0)? t1w*h + t2h*w <= w*h : t1w*h + t2h*w >= w*h);
    }
    @Override
    boolean collides(Point self, Obj o)
    {
        Point a = new Point (self.x+w,self.y); // x축 변화
        Point c = new Point (self.x,self.y+h); // y축 변화
        
        if(collides(self,o.p)) return true; // 상대 도형의 중심이나 들어와 있다면 충돌
            
        if (o.collides(self) || o.collides(a) || o.collides(c) ) return true; // 각 꼭지점 충돌 검사
        
        int maxX = Math.max(self.x, self.x+w),minX = Math.min(self.x, self.x+w); 
        int maxY = Math.max(self.y, self.y+h),minY = Math.min(self.y, self.y+h);
            
        // 축에 평행한 변의 점들에 대해 충돌검사
        for(int x=minX;x<=maxX;x++) if(o.collides(new Point(x,self.y))) return true;
        for(int y=minY;y<=maxY;y++) if(o.collides(new Point(self.x,y))) return true;
        
        // 가로 세로 중 더 긴 쪽을 기준으로 1씩 변화시키며 빗변을 따라 충돌검사
        if( Math.abs(w) >= Math.abs(h) ) { 
            for(int x=minX; x<=maxX; x++) // 가로가 길면 x값 1씩 변화
                if(o.collides(new Point(x, a.y + (int)Math.round(h*(a.x-x)/w))))
                    return true;
        } else {
            for(int y=minY; y<=maxY; y++) // 세로가 길면 y값 1씩 변화
                if(o.collides(new Point(c.x + (int)Math.round(w*(c.y-y)/h), y)))
                    return true;
        }
        
        return false;
    }    
        
    @Override
    String toSVG(Point self) { 
        return "<polygon points='"+self.x+","+self.y+" "+ (self.x+w)+","+self.y +" "+self.x+","+(self.y +h)+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }

}

com.twosigma.beaker.javash.bkrcb76a6d3.RTriangle

In [44]:
import com.twosigma.beakerx.widget.*;
String s= TestData.Vaild();
HTML widget = new HTML();
widget.setValue("<div>"+s+"</div>");
return widget;

In [45]:
class TestDataObj {
    public static String Vaild(){
        int count = 5;
        java.util.Random r = new java.util.Random();
        RGB red = new RGB(255,0,0);
        RGB blue = new RGB(0,0,255);

        String sgvs="";
        for(int i=0; i<count;i++)
        {
            CircObj o1 = new CircObj(r.nextInt(95)+5, new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), red);
            CircObj o2 = new CircObj(r.nextInt(95)+5, new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), blue);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>"+ 
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>"
            ;
        }
        for(int i=0; i<count;i++)
        {
            RectObj o1 = new RectObj(r.nextInt(190)+10, r.nextInt(190)+10, new Point( r.nextInt(100)+50, r.nextInt(100)+50 ), red);
            RectObj o2 = new RectObj(r.nextInt(190)+10, r.nextInt(190)+10, new Point( r.nextInt(100)+50, r.nextInt(100)+50 ), blue);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>"+
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>";
        }
        for(int i=0; i<count;i++)
        {
            RTriObj o1 = new RTriObj(r.nextInt(200)-100, r.nextInt(200)-100, new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), red);
            RTriObj o2 = new RTriObj(r.nextInt(200)-100, r.nextInt(200)-100, new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), blue);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>" +
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>";
           
        }
        for(int i=0; i<10;i++)
        {
            RTriObj o1 = new RTriObj(r.nextInt(200)-100, r.nextInt(200)-100,new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), red);
            RectObj o2 = new RectObj(r.nextInt(190)+10, r.nextInt(190)+10, new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), blue);
            CircObj o3 = new CircObj(r.nextInt(95)+5, new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), new RGB(0,255,0) );

            
            
            sgvs +=
            "<br/><br/><p>"+ i +" 삼각형<->사각형:" + o1.collides(o2)+","+o2.collides(o1)+
            "<p>    사각형<->원형:" + o2.collides(o3)+","+o3.collides(o2)+
            "<p>    원형<->삼각형:" + o3.collides(o1)+","+o1.collides(o3)+ 
            "</p>"+  
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 + o3 + //o4 +
            "</svg>";

        }
        sgvs += ""+sgvs.contains("false,true") +sgvs.contains("true,false");
         return sgvs;   
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.TestDataObj

In [46]:
class TestData {
    public static String Vaild(){
        int count = 5;
        java.util.Random r = new java.util.Random();
        RGB red = new RGB(255,0,0);
        RGB blue = new RGB(0,0,255);

        String sgvs="";
        for(int i=0; i<count;i++)
        {
            Circle t1 = new Circle( r.nextInt(95)+5, red );
            Circle t2 = new Circle( r.nextInt(95)+5,blue );
            Obj o1 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t2);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>"+ 
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>"
            ;
        }
        for(int i=0; i<count;i++)
        {
            Rectangle t1 = new Rectangle( r.nextInt(190)+10, r.nextInt(190)+10,red );
            Rectangle t2 = new Rectangle( r.nextInt(190)+10, r.nextInt(190)+10,blue );
            Obj o1 = new Obj(new Point( r.nextInt(100)+50, r.nextInt(100)+50 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+50, r.nextInt(100)+50 ), t2);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>"+
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>";
        }
        for(int i=0; i<count;i++)
        {
            RTriangle t1 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,red );
            RTriangle t2 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,blue );
            Obj o1 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t2);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>" +
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>";
           
        }
        for(int i=0; i<10;i++)
        {
            RTriangle t1 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,red );
            Rectangle t2 = new Rectangle( r.nextInt(190)+10, r.nextInt(190)+10,blue );
            Circle t3 = new Circle( r.nextInt(95)+5,new RGB(0,255,0) );
            Obj o1 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t2);
            Obj o3 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t3);
            
            sgvs +=
            "<br/><br/><p>"+ i +" 삼각형<->사각형:" + o1.collides(o2)+","+o2.collides(o1)+
            "<p>    사각형<->원형:" + o2.collides(o3)+","+o3.collides(o2)+
            "<p>    원형<->삼각형:" + o3.collides(o1)+","+o1.collides(o3)+ 
            "</p>"+  
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 + o3 + //o4 +
            "</svg>";

        }
        sgvs += ""+sgvs.contains("false,true") +sgvs.contains("true,false");
         return sgvs;   
    }
}

com.twosigma.beaker.javash.bkrcb76a6d3.TestData